# [Day 11](https://adventofcode.com/2022/day/11)

## Read input file

In [1]:
with open('input.txt', 'r') as file:
    lines = [line.strip() for line in file.read().strip().split('\n')]

## Part 1

In [2]:
import numpy
import re

class Monkey:
    def __new__(cls, id):
        monkey = super().__new__(cls)
        monkey.id = id
        monkey.items = list()
        monkey.op_value = None
        monkey.op = None
        monkey.mod_test = None
        monkey.monkey_true = None
        monkey.monkey_false = None
        monkey.inspections = 0
        return monkey

class Monkeys:
    
    def __init__(self):
        self.monkeys = dict()
        
    def add_monkey(self, id):
        self.monkeys[id] = Monkey(id)
        
    def monkey_turn(self, monkey):
        if monkey.op == '*':
            if monkey.op_value == 'old':
                monkey.items = [item*item for item in monkey.items]
            else:
                monkey.items = [item*monkey.op_value for item in monkey.items]
        elif monkey.op == '+':
            if monkey.op_value == 'old':
                monkey.items = [item+item for item in monkey.items]
            else:
                monkey.items = [item+monkey.op_value for item in monkey.items]
        
        monkey.items = [int(item/3.0) for item in monkey.items]

        for item in monkey.items:
            if item % monkey.mod_test:
                self.monkeys[monkey.monkey_false].items.append(item)
            else:
                self.monkeys[monkey.monkey_true].items.append(item)
        monkey.inspections += len(monkey.items)
        monkey.items.clear()
    
    def turn(self):
        for id in sorted(self.monkeys):
            self.monkey_turn(self.monkeys[id])

curr = None
monkeys = Monkeys()
for line in lines:
    if m := re.match(r'Monkey ([0-9]+):', line):
        curr = int(m.group(1))
        monkeys.add_monkey(curr)
    if m := re.match(r'Starting items: (.+)', line):
        monkeys.monkeys[curr].items = [int(item.strip()) for item in m.group(1).split(',')]
    if m := re.match(r'Operation: new = old ([+\*]) (.+)', line):
        monkeys.monkeys[curr].op = m.group(1)
        op_value = m.group(2)
        if op_value != 'old':
            op_value = int(op_value)
        monkeys.monkeys[curr].op_value = op_value
    if m := re.match(r'Test: divisible by ([0-9]+)', line):
        monkeys.monkeys[curr].mod_test = int(m.group(1))
    if m := re.match(r'If true: throw to monkey ([0-9]+)', line):
        monkeys.monkeys[curr].monkey_true = int(m.group(1))
    if m := re.match(r'If false: throw to monkey ([0-9]+)', line):
        monkeys.monkeys[curr].monkey_false = int(m.group(1))

for i in range(20):
    monkeys.turn()

top_inspections = sorted([monkey.inspections for monkey in monkeys.monkeys.values()])[-2:]
display(top_inspections)
display(f'Level: {top_inspections[0] * top_inspections[1]}')

[347, 350]

'Level: 121450'

## Part 2

In [3]:
class BigMonkeys:
    
    def __init__(self):
        self.monkeys = dict()
        
    def add_monkey(self, id):
        self.monkeys[id] = Monkey(id)
        self.monkeys[id].remainders = None
        
    def monkey_turn(self, monkey):
        
        for remainders in monkey.remainders:
            for div, remainder in remainders.items():
                if monkey.op == '*':
                    if monkey.op_value == 'old':
                        remainders[div] = (remainder*remainder) % div
                    else:
                        remainders[div] = (remainder*monkey.op_value) % div
                elif monkey.op == '+':
                    if monkey.op_value == 'old':
                        remainders[div] = (remainder+remainder) % div
                    else:
                        remainders[div] = (remainder+monkey.op_value) % div

        for i, remainders in enumerate(monkey.remainders):
            target = self.monkeys[monkey.monkey_true]
            if remainders[monkey.mod_test]:
                target = self.monkeys[monkey.monkey_false]
            target.items.append(monkey.items[i])
            target.remainders.append(remainders)
        monkey.inspections += len(monkey.items)
        monkey.items.clear()
        monkey.remainders.clear()
    
    def turn(self):
        for monkey in self.monkeys.values():
            if monkey.remainders is None:
                monkey.remainders = list()
                for item in monkey.items:
                    remainder = dict()
                    for m in self.monkeys.values():
                        remainder[m.mod_test] = item % m.mod_test
                    monkey.remainders.append(remainder) 
                    
        for id in sorted(self.monkeys):
            self.monkey_turn(self.monkeys[id])
            
curr = None
monkeys = BigMonkeys()
for line in lines:
    if m := re.match(r'Monkey ([0-9]+):', line):
        curr = int(m.group(1))
        monkeys.add_monkey(curr)
    if m := re.match(r'Starting items: (.+)', line):
        monkeys.monkeys[curr].items = [int(item.strip()) for item in m.group(1).split(',')]
    if m := re.match(r'Operation: new = old ([+\*]) (.+)', line):
        monkeys.monkeys[curr].op = m.group(1)
        op_value = m.group(2)
        if op_value != 'old':
            op_value = int(op_value)
        monkeys.monkeys[curr].op_value = op_value
    if m := re.match(r'Test: divisible by ([0-9]+)', line):
        monkeys.monkeys[curr].mod_test = int(m.group(1))
    if m := re.match(r'If true: throw to monkey ([0-9]+)', line):
        monkeys.monkeys[curr].monkey_true = int(m.group(1))
    if m := re.match(r'If false: throw to monkey ([0-9]+)', line):
        monkeys.monkeys[curr].monkey_false = int(m.group(1))


for i in range(10000):
    monkeys.turn()

top_inspections = sorted([monkey.inspections for monkey in monkeys.monkeys.values()])[-2:]
display(top_inspections)
display(f'Level: {top_inspections[0] * top_inspections[1]}')

[166922, 169205]

'Level: 28244037010'